In [ ]:
from coldquanta.qiskit_tools.coldquanta_sim_backend import ColdQuantaSimBackend
%load_ext autoreload
%autoreload 2

rows = 2
cols = 2
neutral_atom = ColdQuantaSimBackend(rows=rows, cols=cols)
neutral_atom_config = neutral_atom.configuration()
coupling_map = neutral_atom_config.coupling_map

In [ ]:
from networkx import Graph, draw_planar
g = Graph(coupling_map)
draw_planar(g)

In [ ]:
import numpy as np

np.random.seed(137)
weights = set()
for k in coupling_map:
    weights.add((min(k), max(k)))
# Fix the weights to 1, for now.
weights = {k: 1  for k in weights}

In [ ]:
from classical_optimization.qaoa_circuits import maxcut_qaoa_circuit

In [ ]:
print(maxcut_qaoa_circuit(gammas=[0,0],betas=[0,0], p=1, weights=weights, rows=rows, cols=cols))

In [ ]:
def produce_gammas_betas(discretization, max_gamma, max_beta):
    return np.linspace(0, max_gamma, discretization), np.linspace(0, max_beta, discretization)

discretization = 20
max_gamma = max_beta = np.pi
gammas, betas = produce_gammas_betas(discretization, max_gamma, max_beta)

In [ ]:
from qiskit import Aer, execute
simulator = Aer.get_backend('qasm_simulator')
num_shots = 1024
experiments = []
for gamma in gammas:
    for beta in betas:
        experiments.append(maxcut_qaoa_circuit(gammas=[gamma], betas=[beta], p=1, rows=rows, cols=cols, weights=weights))
job = execute(experiments, backend=simulator, shots=num_shots)    
all_counts = job.result().get_counts()

In [ ]:
from classical_optimization.qaoa_circuits import estimate_cost

landscape = np.zeros((discretization, discretization))
for i, gamma in enumerate(gammas):
    for j, beta in enumerate(betas):
        landscape[i][j] = estimate_cost(all_counts[i*len(betas) + j], weights)

In [ ]:
from classical_optimization.qaoa_circuits import plot_landscape

plot_landscape(landscape, max_gamma, max_beta, colorbar=True)

In [ ]:
from qiskit.visualization import plot_histogram

plot_histogram(all_counts[np.argmax(landscape)])

# Now introduce noise!

In [ ]:
from coldquanta.qiskit_tools.modeling.neutral_atom_noise_model import create_noise_model

In [ ]:
noise_model = create_noise_model()

In [ ]:
noise_model = create_noise_model(cz_fidelity=0.7)

experiments = []
for gamma in gammas:
    for beta in betas:
        experiments.append(maxcut_qaoa_circuit(gammas=[gamma], betas=[beta], p=1, rows=rows, cols=cols, weights=weights))
job = execute(experiments,
              simulator,
              noise_model=noise_model,
              coupling_map=coupling_map,
              optimization_level=0,
              shots=num_shots)  
all_counts = job.result().get_counts()

In [ ]:
noisy_landscape = np.zeros((discretization, discretization))
for i, gamma in enumerate(gammas):
    for j, beta in enumerate(betas):
        noisy_landscape[i][j] = estimate_cost(all_counts[i*len(betas) + j], weights)

In [ ]:
plot_landscape(noisy_landscape, max_gamma, max_beta);

In [ ]:
plot_histogram(all_counts[np.argmax(noisy_landscape)])

# Include Classical Optimizer

In [ ]:
from classical_optimization.qaoa_circuits import execute_qaoa_circuit_and_estimate_cost
simulator = Aer.get_backend('qasm_simulator')
shots_per_point = 1000

In [ ]:
from classical_optimization.qaoa_circuits import try_optimizer
import matplotlib.pyplot as plt

for method in ['nelder-mead', 'powell']:
    history = []
    result = try_optimizer(method, simulator, neutral_atom_config.coupling_map, shots_per_point,
                           weights, max_gamma, max_beta, rows, cols, history)
    history.append(result.x) #We will append the last visited point, since the plotting assumes this is the optimal value.
    plot_landscape(landscape, max_gamma, max_beta, history=history, discretization=discretization)
    plt.title(method)
    plt.show()

## Larger Depths!

In [ ]:
discretization = 8
max_gamma = max_beta = np.pi
gammas, betas = produce_gammas_betas(discretization, max_gamma, max_beta)

In [ ]:
import tqdm as tqdm
from qiskit import Aer, execute
simulator = Aer.get_backend('qasm_simulator')
num_shots = 1024
experiments = []
for gamma in gammas:
    for beta in betas:
        for gamma1 in gammas:
            for beta1 in betas:
                experiments.append(maxcut_qaoa_circuit(gammas=[gamma, gamma1], betas=[beta, beta1], p=2, rows=rows, cols=cols, weights=weights))
job = execute(experiments, backend=simulator, shots=num_shots)    
all_counts = job.result().get_counts()

In [ ]:
landscape = np.zeros((discretization, discretization, discretization, discretization))
for i, gamma in enumerate(gammas):
    for j, beta in enumerate(betas):
        for k, gamma in enumerate(gammas):
            for l, beta in enumerate(betas):
                landscape[i][j][k][l] = estimate_cost(all_counts[i*len(betas)*len(gammas)*len(betas)+j*len(betas)*len(gammas)+k*len(betas) + l], weights)

In [ ]:
plot_landscape(landscape[np.unravel_index(landscape.argmax(), landscape.shape)[0:2]], max_gamma, max_beta, colorbar=True)

In [ ]:
plot_histogram(all_counts[landscape.argmax()])